In [1]:
# Import Dependencies
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm

In [2]:
# Load AQI CSV
aqi_path = ('../Resources/max_aqi_csv.csv')

#Read in the AQI CSV and store into a Pandas DF
aqi_df =pd.read_csv(aqi_path)
aqi_df.head()

,State,State Name,Ozone,NO2,SO2,CO,PM 2.5
0,1,Alabama,0.045526,22.978000,4.673446,0.345078,10.016438
1,4,Arizona,0.057240,38.920330,12.117582,0.684723,17.213333
2,5,Arkansas,0.043364,17.477596,1.180055,0.287533,9.809836
3,6,California,0.074551,43.779130,3.285256,0.620842,40.764417
4,8,Colorado,0.065656,43.903683,2.054972,0.437755,10.486665


In [3]:
# Remove extra index
aqi_df = aqi_df[['State', 'State Name', 'Ozone', 'NO2', 'SO2', 'CO', 'PM 2.5']]
aqi_df.head()

,State,State Name,Ozone,NO2,SO2,CO,PM 2.5
0,1,Alabama,0.045526,22.978000,4.673446,0.345078,10.016438
1,4,Arizona,0.057240,38.920330,12.117582,0.684723,17.213333
2,5,Arkansas,0.043364,17.477596,1.180055,0.287533,9.809836
3,6,California,0.074551,43.779130,3.285256,0.620842,40.764417
4,8,Colorado,0.065656,43.903683,2.054972,0.437755,10.486665


In [5]:
# Load CDI CSV
cdi_path = ('../Resources/cdi_csv.csv')

#Read in the CDI CSV and store into a Pandas DF
cdi_df = pd.read_csv(cdi_path)

In [6]:
# Remove extra index
cdi_df = cdi_df.drop('Unnamed: 0', axis=1)
cdi_df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,Topic,Question,DataValueType,DataValue,DataValueAlt,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1,LocationID,Latitude,Longitude
0,2020,2020,NY,New York,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,6.0,6.0,4.2,8.5,Race/Ethnicity,"Other, non-Hispanic",36,-75.543970,42.827001
1,2020,2020,OH,Ohio,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,20.7,20.7,14.9,28.0,Race/Ethnicity,"Multiracial, non-Hispanic",39,-82.404260,40.060210
2,2020,2020,PA,Pennsylvania,Asthma,Current asthma prevalence among adults aged >=...,Crude Prevalence,16.0,16.0,11.1,22.7,Race/Ethnicity,Hispanic,42,-77.860700,40.793730
3,2020,2020,OR,Oregon,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,16.8,16.8,9.8,27.1,Race/Ethnicity,"Multiracial, non-Hispanic",41,-120.155031,44.567449
4,2020,2020,AR,Arkansas,Asthma,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.2,9.2,8.0,10.5,Race/Ethnicity,"White, non-Hispanic",5,-92.274491,34.748650


In [7]:
# Limit to rows where stratification equals overall because otherwise the grouped data will be incorrect
overall_only = cdi_df[cdi_df['StratificationCategory1'] == 'Overall']


In [8]:
# Limit to relevent columns for readability
overall_df = overall_only[["LocationDesc", "Topic", "Question", "DataValueType", "DataValue", "LowConfidenceLimit", "HighConfidenceLimit"]]

In [9]:
# Sort by state to ensure we've got all of the required data
overall_sorted = overall_df.sort_values(by=['LocationDesc', 'Topic'])

# Remove rows where DataValueType is 'Number' because that is not a helpful indicator here

overall_cleaned = overall_sorted[(overall_sorted['DataValueType'] != 'Number') & 
                                 (overall_sorted['DataValueType'] != 'Crude Rate') &
                                 (overall_sorted['DataValueType'] != 'Crude Prevalence')]

# Reset the index
overall_cleaned = overall_cleaned.reset_index(drop=True)
overall_cleaned.head(20)

,LocationDesc,Topic,Question,DataValueType,DataValue,LowConfidenceLimit,HighConfidenceLimit
0,Alabama,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.0,8.0,10.2
1,Alabama,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,312.5,308.1,317.0
2,Alabama,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,8.9,7.9,10.0
3,Alaska,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,8.8,7.4,10.5
4,Alaska,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,191.1,180.0,202.2
5,Alaska,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,4.6,3.8,5.5
6,Arizona,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.7,8.9,10.5
7,Arizona,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,195.9,193.1,198.8
8,Arizona,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,5.4,4.8,6.0
9,Arkansas,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,8.9,7.8,10.2


In [10]:
## Change the name of the "LocationDesc" column to match "State Name" in AQI data
overall_cleaned = overall_cleaned.rename(columns={'LocationDesc': 'State Name'})

In [11]:
## Merge AQI and CDI DFs on State Name
aqi_cdi_df = pd.merge(overall_cleaned, aqi_df, on='State Name', how='left')

# Drop State Code column
aqi_cdi_df = aqi_cdi_df.drop('State', axis=1)
aqi_cdi_df.head()

,State Name,Topic,Question,DataValueType,DataValue,LowConfidenceLimit,HighConfidenceLimit,Ozone,NO2,SO2,CO,PM 2.5
0,Alabama,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.0,8.0,10.2,0.045526,22.978,4.673446,0.345078,10.016438
1,Alabama,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,312.5,308.1,317.0,0.045526,22.978,4.673446,0.345078,10.016438
2,Alabama,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,8.9,7.9,10.0,0.045526,22.978,4.673446,0.345078,10.016438
3,Alaska,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,8.8,7.4,10.5,NaN,NaN,NaN,NaN,NaN
4,Alaska,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,191.1,180.0,202.2,NaN,NaN,NaN,NaN,NaN


In [12]:
## Create a DF without missing AQI data for the purpose of creating scatter plots
complete_aqi_cdi = aqi_cdi_df.dropna()
complete_aqi_cdi.head(50)

,State Name,Topic,Question,DataValueType,DataValue,LowConfidenceLimit,HighConfidenceLimit,Ozone,NO2,SO2,CO,PM 2.5
0,Alabama,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.0,8.0,10.2,0.045526,22.978000,4.673446,0.345078,10.016438
1,Alabama,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,312.5,308.1,317.0,0.045526,22.978000,4.673446,0.345078,10.016438
2,Alabama,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,8.9,7.9,10.0,0.045526,22.978000,4.673446,0.345078,10.016438
6,Arizona,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.7,8.9,10.5,0.057240,38.920330,12.117582,0.684723,17.213333
7,Arizona,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,195.9,193.1,198.8,0.057240,38.920330,12.117582,0.684723,17.213333
8,Arizona,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,5.4,4.8,6.0,0.057240,38.920330,12.117582,0.684723,17.213333
9,Arkansas,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,8.9,7.8,10.2,0.043364,17.477596,1.180055,0.287533,9.809836
10,Arkansas,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,284.1,278.7,289.5,0.043364,17.477596,1.180055,0.287533,9.809836
11,Arkansas,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,8.1,7.2,9.0,0.043364,17.477596,1.180055,0.287533,9.809836
12,California,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.3,8.2,10.5,0.074551,43.779130,3.285256,0.620842,40.764417


In [13]:
# Create a CSV merging the data together
complete_aqi_cdi.to_csv('../Resources/aqi_cdi.csv')

In [14]:
## Create a DF with only COPD data
copd_df = complete_aqi_cdi[complete_aqi_cdi["Topic"] == 'Chronic Obstructive Pulmonary Disease']
copd_df.head()

,State Name,Topic,Question,DataValueType,DataValue,LowConfidenceLimit,HighConfidenceLimit,Ozone,NO2,SO2,CO,PM 2.5
2,Alabama,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,8.9,7.9,10.0,0.045526,22.978000,4.673446,0.345078,10.016438
8,Arizona,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,5.4,4.8,6.0,0.057240,38.920330,12.117582,0.684723,17.213333
11,Arkansas,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,8.1,7.2,9.0,0.043364,17.477596,1.180055,0.287533,9.809836
14,California,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,5.1,4.3,6.0,0.074551,43.779130,3.285256,0.620842,40.764417
17,Colorado,Chronic Obstructive Pulmonary Disease,Prevalence of chronic obstructive pulmonary di...,Age-adjusted Prevalence,4.3,3.9,4.7,0.065656,43.903683,2.054972,0.437755,10.486665


In [15]:
## Create a DF with only asthma data
asthma_df = complete_aqi_cdi[complete_aqi_cdi["Topic"] == 'Asthma']
asthma_df.head()

,State Name,Topic,Question,DataValueType,DataValue,LowConfidenceLimit,HighConfidenceLimit,Ozone,NO2,SO2,CO,PM 2.5
0,Alabama,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.0,8.0,10.2,0.045526,22.978000,4.673446,0.345078,10.016438
6,Arizona,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.7,8.9,10.5,0.057240,38.920330,12.117582,0.684723,17.213333
9,Arkansas,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,8.9,7.8,10.2,0.043364,17.477596,1.180055,0.287533,9.809836
12,California,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.3,8.2,10.5,0.074551,43.779130,3.285256,0.620842,40.764417
15,Colorado,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,9.6,8.9,10.3,0.065656,43.903683,2.054972,0.437755,10.486665


In [16]:
## Create a DF with only CVD data
cvd_df = complete_aqi_cdi[complete_aqi_cdi["Topic"] == 'Cardiovascular Disease']
cvd_df.head()

,State Name,Topic,Question,DataValueType,DataValue,LowConfidenceLimit,HighConfidenceLimit,Ozone,NO2,SO2,CO,PM 2.5
1,Alabama,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,312.5,308.1,317.0,0.045526,22.978000,4.673446,0.345078,10.016438
7,Arizona,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,195.9,193.1,198.8,0.057240,38.920330,12.117582,0.684723,17.213333
10,Arkansas,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,284.1,278.7,289.5,0.043364,17.477596,1.180055,0.287533,9.809836
13,California,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,202.7,201.4,204.0,0.074551,43.779130,3.285256,0.620842,40.764417
16,Colorado,Cardiovascular Disease,Mortality from total cardiovascular diseases,Age-adjusted Rate,178.6,175.2,182.0,0.065656,43.903683,2.054972,0.437755,10.486665


In [17]:
## Ozone Scatter Plot

ozone_scatter = px.scatter(complete_aqi_cdi, 
                           x='DataValue', y='Ozone', 
                           color='Topic',
                           hover_name = "State Name",
                           title = 'Effect of Ozone on CVD, COPD, and Asthma',
                           labels = {'DataValue': 'Age-adjusted CDI Data', 'Ozone': 'Annual Ozone Values'})
ozone_scatter.show()

In [18]:
## Define function to plot one parameter against one outcome
def plot_indiv_cdi(parameter, outcome, df):
    scatter = px.scatter(df, 
                           x='DataValue', y=parameter, 
                           hover_name = "State Name",
                           title = f'Effect of {parameter} on {outcome}',
                           labels = {'DataValue': 'Age-adjusted CDI Data', parameter: f'Annual {parameter} Values'},
                           trendline = 'ols')
    
    # Fit linear regression model
    X = df['DataValue']
    y = df[parameter]
    X = sm.add_constant(X)  # Add constant term for intercept
    model = sm.OLS(y, X).fit()

    r_squared = model.rsquared

    scatter.update_layout(title=f'Effect of {parameter} on {outcome}<br>R-squared: {r_squared:.2f}')
    

    scatter.show()


In [19]:
## Ozone and CVD
plot_indiv_cdi('Ozone', "CVD", cvd_df)

In [20]:
## Ozone and COPD
plot_indiv_cdi('Ozone', "COPD", copd_df)

In [21]:
## Ozone and Asthma
plot_indiv_cdi('Ozone', "Asthma", asthma_df)

In [22]:
## NO2 and CVD
plot_indiv_cdi('NO2', "CVD", cvd_df)

In [23]:
## NO2 and COPD
plot_indiv_cdi('NO2', "COPD", copd_df)

In [24]:
## NO2 and Asthma
plot_indiv_cdi('NO2', "Asthma", asthma_df)

In [25]:
plot_indiv_cdi('SO2', "CVD", cvd_df)

In [26]:
plot_indiv_cdi('SO2', "COPD", copd_df)

In [27]:
plot_indiv_cdi('SO2', "Asthma", asthma_df)

In [28]:
plot_indiv_cdi('CO', "CVD", cvd_df)

In [29]:
plot_indiv_cdi('CO', "COPD", copd_df)

In [30]:
plot_indiv_cdi('CO', "Asthma", asthma_df)

In [31]:
plot_indiv_cdi('PM 2.5', "CVD", cvd_df)

In [32]:
plot_indiv_cdi('PM 2.5', "COPD", copd_df)

In [33]:
plot_indiv_cdi('PM 2.5', "Asthma", asthma_df)